In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sklearn

In [ ]:
os.getcwd()

In [ ]:
os.chdir(r"D:\insurance-premium-prediction\insurance\artifact\data_ingestion\2022-07-28-19-15-12\ingested_data\train")

In [ ]:
os.getcwd()

In [ ]:
df = pd.read_csv("insurance.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df[df.duplicated() == True]

In [ ]:
df = df.drop_duplicates()

In [ ]:
# checking if any missing values
df.isnull().sum()

In [ ]:
plt.figure(figsize=(20,12))
df.hist()

Observation:
1. age is not normally distributed.
2. bmi is noramlly distributed.
3. most of the samples have no children
4. expenses is following right skewed distribution.

In [ ]:
categorical_columns = []
for column in df.columns:
    if df[column].dtype == 'object':
        categorical_columns.append(column)
categorical_columns

In [ ]:
plt.figure(figsize=(10,10))
for i,col in enumerate(categorical_columns):
    ax = plt.subplot(3,1,i+1)
    sns.countplot(x=col, data=df, ax=ax)
plt.tight_layout()
    
        

Observation:

1.male and female are having equal count in sex column.
2.no smoking count is more than 800 as compared to 200 smoking.
3.southeast region is having more count followed by northeast, northwest and southwest.

In [ ]:
sns.relplot(x='age', y='expenses', data=df, hue='smoker', col='sex', size='bmi')

In [ ]:
sns.relplot(x='age', y='expenses', data=df, hue='region', col='sex', size='bmi')

In [ ]:
sns.lmplot(x='age', y='expenses',hue='smoker', data=df)

Observation:

1.people who are non-smoker are having expenses less than 30k and more samples ranges between 2k to 15k.
2.people who smokes are having expenses more than 15k and more samples ranges between 30k to 50k.
3.age increases as expenses increases.

In [ ]:
sns.relplot(x='bmi', y='expenses', data=df, hue='smoker', col='sex', size='age')

In [ ]:
sns.relplot(x='bmi', y='expenses', data=df, hue='region', col='sex', size='age')

In [ ]:
sns.lmplot(x='bmi', y='expenses',hue='smoker', data=df)

Observation:
1.some linear relatioship we can see between expenses and bmi.
2.non-smokers samples are more populated under 15K whereas smokers above 20K.

In [ ]:
sns.catplot(x='smoker', y='expenses', data=df)

Observation:
1.Smokers expenses are above 15k.
2.Non-smokers expenses are below 35k.

In [ ]:
numerical_columns = df.select_dtypes('number').columns
numerical_columns

In [ ]:
plt.figure(figsize=(12,10))
for i, col in enumerate(numerical_columns):
    ax = plt.subplot(4,1,i+1)
    sns.boxplot(x=col, data=df, ax=ax)
plt.tight_layout()

Observation:
1.Outiliers are found in BMI and Expenses.
2.bmi is following normal distribution and expenses is right skewed distribution.

In [ ]:
sns.boxplot(x='bmi', y='smoker',data=df, hue='sex')

In [ ]:
upper_limit = df['bmi'].mean() + 3*df['bmi'].std()
lower_limit = df['bmi'].mean() - 3*df['bmi'].std()
upper_limit, lower_limit
# calculating upper and lower limit for bmi attribute

In [ ]:
df[df['bmi'] > upper_limit]
df.loc[df['bmi'] > upper_limit, 'bmi'] = upper_limit
# replacing outliers with upper_limit

In [ ]:
df.head()

In [ ]:
# correlation
sns.heatmap(df.corr(),annot=True)
# age and bmi are correlated with target column whereas children is having very less correlation.